In [5]:
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools
from itertools import combinations

import random
import joblib
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

from pd.params import *



In [ ]:
def preprocess_data(data_type="train"):
    
    if data_type == "train":
        data = pd.read_parquet(OUTDIR+"train_data.parquet")
        train_labels = pd.read_csv(DATADIR+"train_labels.csv")
    else:
        data = pd.read_parquet(OUTDIR+"test_data.parquet")
        train_labels = None
    
    print('Starting feature engineer...')
    
    data_cont_agg = data.groupby("customer_ID")[ContCols].agg(['mean', 'std', 'min', 'max', 'last'])
    data_cont_agg.columns = ['_'.join(x) for x in data_cont_agg.columns]
    data_cont_agg.reset_index(inplace=True)

    data_cat_agg = data.groupby("customer_ID")[CATCOLS].agg(['count', 'last', 'nunique'])
    data_cat_agg.columns = ['_'.join(x) for x in data_cat_agg.columns]
    data_cat_agg.reset_index(inplace=True)
    data = data_cont_agg.merge(data_cat_agg, how='inner', on='customer_ID')
    
    if train_labels is None:
        data = data_cont_agg.merge(train_labels, how='inner', on='customer_ID')
    
    del data_cont_agg, data_cont_agg
    gc.collect()
    
    data.to_parquet(OUTDIR+f"{data_type}_fe.parquet")

In [2]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True


In [9]:
params = {
        'objective': 'binary',
        'metric': "binary_logloss",
        'boosting': 'dart',
        'seed': 42,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 4,
        'min_data_in_leaf': 40
        }

In [5]:
train_data = np.load(OUTDIR+"train_data_all.npy").transpose((0, 2, 1))
train_labels = np.load(OUTDIR+"train_labels_all.npy")


In [6]:
train_data = np.load(OUTDIR+"c13_data.npy")
train_labels = np.load(OUTDIR+"c13_labels.npy")

In [ ]:
X_train[:, :, 1].reshape(X_train.shape[0], 13, -1).shape  

In [12]:

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=1/9, random_state=0, shuffle=True)
validation_data = (X_test, y_test)


In [13]:
lgb_train = lgb.Dataset(X_train.reshape(X_train.shape[0], -1), y_train)
lgb_valid = lgb.Dataset(X_test.reshape(X_test.shape[0], -1), y_test,)
model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 100,
            verbose_eval = 100,
            feval = lgb_amex_metric
            )
        

[LightGBM] [Info] Number of positive: 79538, number of negative: 263603
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.031721 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 328485
[LightGBM] [Info] Number of data points in the train set: 343141, number of used features: 2424
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.231794 -> initscore=-1.198209
[LightGBM] [Info] Start training from score -1.198209
[100]	training's binary_logloss: 0.447318	training's amex_metric: 0.78539	valid_1's binary_logloss: 0.449321	valid_1's amex_metric: 0.76945
[200]	training's binary_logloss: 0.430475	training's amex_metric: 0.790134	valid_1's binary_logloss: 0.43309	valid_1's amex_metric: 0.772277
[300]	training's binary_logloss: 0.37635	training's amex_metric: 0.794047	valid_1's binary_logloss: 0.380053	valid_1's amex_metric: 0.77517
[400]	traini

In [ ]:
def train_and_evaluate(train, test=None, n_folds=5, seed=42):
    cat_features = [f"{cf}_last" for cf in CATCOLS]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        if test is not None:
            test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        if test is not None:
            test[col + '_round2'] = test[col].round(2)
    # Get feature list
    features = [col for col in train.columns if col not in ['customer_ID', "S_2", "target"]]
    
    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train["target"])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train["target"].iloc[trn_ind], train["target"].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
        model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 100,
            verbose_eval = 100,
            feval = lgb_amex_metric
            )
        # Save best model
        joblib.dump(model, OUTDIR+f'Models/lgbm_fold{fold}_seed{seed}.pkl')
        val_pred = model.predict(x_val) # Predict validation
        oof_predictions[val_ind] = val_pred  # Add to out of folds array
        if test is not None:
            test_pred = model.predict(test[features]) # Predict the test set
            test_predictions += test_pred/n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
    score = amex_metric(train["target"], oof_predictions)  # Compute out of folds metric
    print(f'Our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train["target"], 'prediction': oof_predictions})
    oof_df.to_csv(OUTDIR+f'oof_lgbm_baseline_{n_folds}fold_seed{seed}.csv', index=False)
    # Create a dataframe to store test prediction
    if test is not None:
        test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
        test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_lgbm_baseline_{n_folds}fold_seed{seed}.csv', index = False)

In [ ]:
def train_lgbm_single_feature(train, feature, n_folds=5, seed=42):
    cat_feature = "auto"
    if feature in CATCOLS:
        cat_feature = feature
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train["target"])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} of {feature} feature...')
        x_train, x_val = train[feature].iloc[trn_ind], train[feature].iloc[val_ind]
        y_train, y_val = train["target"].iloc[trn_ind], train["target"].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=cat_feature)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature=cat_feature)
        model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 100,
            verbose_eval = 100,
            feval = lgb_amex_metric
            )
        # Save best model
        joblib.dump(model, OUTDIR+f'Models/lgbm_fold{fold}_seed{seed}.pkl')
        val_pred = model.predict(x_val) # Predict validation
        oof_predictions[val_ind] = val_pred  # Add to out of folds array
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
    score = amex_metric(train["target"], oof_predictions)  # Compute out of folds metric
    print(f'Our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train["target"], 'prediction': oof_predictions})
    oof_df.to_csv(OUTDIR+f'oof_lgbm_baseline_{n_folds}fold_seed{seed}.csv', index=False)
    